In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import train_test_split
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.legend_handler import HandlerLine2D
# import json
from pylab import figure
import ast
import os
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
from dtw import dtw
import functools
import math
import itertools
%pylab inline

In [ ]:
def load_dataframe(filepath):
    data = pd.read_csv(filepath, sep=", ", header=None)
    data[0] = data[0].apply(lambda x: x.replace("[(", ""))
    data[2] = data[2].apply(lambda x: x.replace(")", ""))
    data[3] = data[3].apply(lambda x: x.replace("(", ""))
    data[5] = data[5].apply(lambda x: x.replace(")]", ""))
    for i in range(len(data.columns)):
        data[i] = data[i].apply(lambda x: int(x))
    data.columns = ['accX', 'accY', 'accZ', 'gyrX', 'gyrY', 'gyrZ']
    return data 

In [29]:
df_11 = load_dataframe('Exp2/P1-FT/front-1.txt')
df_12 = load_dataframe('Exp2/P1-FT/front-2.txt')
df_13 = load_dataframe('Exp2/P1-FT/front-3.txt')
df_14 = load_dataframe('Exp2/P1-FT/front-4.txt')
df_15 = load_dataframe('Exp2/P1-FT/front-5.txt')

df_21 = load_dataframe('Exp2/P1-LT/left-1.txt')
df_22 = load_dataframe('Exp2/P1-LT/left-2.txt')
df_23 = load_dataframe('Exp2/P1-LT/left-3.txt')
df_24 = load_dataframe('Exp2/P1-LT/left-4.txt')
df_25 = load_dataframe('Exp2/P1-LT/left-4.txt')

df_31 = load_dataframe('Exp2/P1-RT/right-1.txt')
df_32 = load_dataframe('Exp2/P1-RT/right-2.txt')
df_33 = load_dataframe('Exp2/P1-RT/right-3.txt')


print(list(df_25["accX"]))

[-15461, -15458, -15458, -15460, -15460, -15460, -15460, -15459, -15459, -15459, -15459, -15459, -15459, -15459, -15456, -15456, -15456, -15456, -15456, -15456, -15456, -15456, -15456, -15454, -15454, -15468, -15468, -15468, -15468, -15468, -15449, -15449, -15449, -15449, -15449, -15449, -15449, -15442, -15442, -15442, -15442, -15442, -15442, -15520, -15520, -15520, -15520, -15520, -15513, -15513, -15513, -15513, -15513, -15513, -15513, -15500, -15500, -15500, -15500, -15500, -15500, -15489, -15489, -15489, -15489, -15491, -15491, -15491, -15491, -15491, -15491, -15491, -15491, -15488, -15488, -15488, -15488, -15488, -15508, -15508, -15508, -15508, -15508, -15496, -15496, -15496, -15496]


/Users/abishkarchhetri/code/research/GestureiOSData/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [5]:
def create_single_row_df(df):
    df1=df.stack().swaplevel()
    df1.index=df1.index.map('{0[0]}_{0[1]}'.format) 
    df_new = df1.to_frame().T
    c = df_new.shape[1] + 1
    while c <= 2880: # 480 * 6
        df_new[c] = np.nan
        c += 1     
    return df_new

In [6]:
# edit this as per the file name 
labels = {'LT': 1, 'RT': 2, 'BFS': 3, 'LRS': 4, 'RLS': 5, 'FBS': 6}

In [7]:
def gesture_overall_dataframe(root_dir, label):
    final_df = pd.DataFrame()
    label_df = pd.DataFrame()
    root_dir = root_dir
    for fname in os.listdir(root_dir):
        if fname == '.DS_Store':
            continue
        filepath = os.path.join(root_dir, fname)
        df = create_single_row_df(load_dataframe(filepath))
        df['label'] = label
        #dataframes.append(df)
        final_df = final_df.append(pd.DataFrame(df.values.tolist()), ignore_index=True)
    #dataframe = pd.concat([each for each in dataframes],ignore_index=True)    
    return final_df

In [8]:
p1_back = gesture_overall_dataframe('Exp2/P1-BT', 1)
p1_back

/Users/abishkarchhetri/code/research/GestureiOSData/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


0       1       2      3      4     5        6       7       8     \
0  -15568.0 -1626.0 -5116.0  -71.0  214.0   7.0 -15570.0 -1642.0 -5151.0   
1  -15582.0 -1563.0 -5154.0  -57.0  188.0  24.0 -15580.0 -1548.0 -5112.0   
2  -15564.0 -1876.0 -4929.0  -57.0  200.0  41.0 -15556.0 -1862.0 -4955.0   
3  -15530.0 -1834.0 -5055.0  -28.0  187.0   9.0 -15524.0 -1856.0 -5060.0   
4  -15551.0 -1857.0 -5146.0  -36.0  187.0  38.0 -15544.0 -1864.0 -5132.0   
5  -15464.0 -1877.0 -5400.0   41.0   82.0   6.0 -15477.0 -1892.0 -5306.0   
6  -15547.0 -1768.0 -5113.0  -59.0  177.0  21.0 -15548.0 -1738.0 -5133.0   
7  -15552.0 -1541.0 -5167.0   -4.0  157.0  50.0 -15553.0 -1526.0 -5180.0   
8  -15514.0 -1816.0 -5203.0  -40.0  165.0  -2.0 -15525.0 -1820.0 -5213.0   
9  -15511.0 -1768.0 -5193.0 -110.0  161.0  15.0 -15504.0 -1770.0 -5179.0   
10 -15513.0 -1813.0 -5228.0  -33.0  164.0  12.0 -15336.0 -1859.0 -5400.0   

     9     ...  2871  2872  2873  2874  2875  2876  2877  2878  2879  2880  
0   -83.0  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   1.0  
1    24.0  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   1.0  
2   -61.0  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   1.0  
3    -1.0  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   1.0  
4   -34.0  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   1.0  
5    20.0  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   1.0  
6  -123.0  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   1.0  
7    -5.0  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   1.0  
8   -39.0  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   1.0  
9   -78.0  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   1.0  
10  338.0  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   1.0  

[11 rows x 2881 columns]

In [31]:
p1_left = gesture_overall_dataframe('P1-LT', 2)
p1_left

/Users/abishkarchhetri/code/research/GestureiOSData/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


FileNotFoundError: [Errno 2] No such file or directory: 'Exp2/Exp2/P1-LT/left-1.txt'

In [12]:
p1_right = gesture_overall_dataframe('P1-RT', 3)
p1_right

FileNotFoundError: [Errno 2] No such file or directory: 'P1-RT'

In [13]:
p1_front = gesture_overall_dataframe('P1-FT', 4)
p1_front

FileNotFoundError: [Errno 2] No such file or directory: 'P1-FT'

In [14]:
p1_lrs = gesture_overall_dataframe('P1-RLS', 100)
p1_lrs

FileNotFoundError: [Errno 2] No such file or directory: 'P1-RLS'

In [15]:
def get_dataframe_for_each_participant(participant_number):
    dfs = []
    for l in labels:
        gesture_name = participant_number + "-" + l
        # Temporary adjustment 
        if (gesture_name == 'P4-LRS') or (gesture_name == 'P4-RLS'):
            continue
        dfs.append(gesture_overall_dataframe(gesture_name, labels[l]))
    df =  pd.concat([each for each in dfs],ignore_index=True)      
    return df

In [16]:
get_dataframe_for_each_participant("P1") #P1, P2, P8, 

FileNotFoundError: [Errno 2] No such file or directory: 'P1-LT'

In [17]:
# get_dataframe_for_each_participant("P3")

In [18]:
def get_dataframe_for_all_participants():
    particiapnts = ['P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12']#['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12']
    dfs = []
    for p in particiapnts:
        dfs.append(get_dataframe_for_each_participant(p))
    df = pd.concat([each for each in dfs],ignore_index=True)      
    return df        

In [19]:
sample_dataframe = get_dataframe_for_all_participants()
sample_dataframe

FileNotFoundError: [Errno 2] No such file or directory: 'P3-LT'

In [20]:
bb = sample_dataframe.iloc[[0],:-1]
bb

NameError: name 'sample_dataframe' is not defined

In [21]:
bb_c = pd.DataFrame(bb.values.reshape(480,-1),
                      index=bb.index.repeat(480))

NameError: name 'bb' is not defined

In [22]:
bb_c

NameError: name 'bb_c' is not defined

In [23]:
bb_cc = bb_c.fillna(727199802211965)
bb_cc

NameError: name 'bb_c' is not defined

In [24]:
bb_ccc = bb_cc.replace(727199802211965, np.nan).dropna()

NameError: name 'bb_cc' is not defined

In [25]:
bb_ccc

NameError: name 'bb_ccc' is not defined

In [26]:
sad = sample_dataframe.copy()

NameError: name 'sample_dataframe' is not defined

In [27]:
sad = sad.fillna(727199802211965)
sad

NameError: name 'sad' is not defined

In [28]:
ddd = pd.DataFrame(sad.iloc[[0],:-1].values)
pd.DataFrame(ddd.values.reshape(480,-1),
                      index=ddd.index.repeat(480))

NameError: name 'sad' is not defined

In [ ]:
def get_dataframe_column_for_comparison(df):
    df = pd.DataFrame(df).T
    dfc = pd.DataFrame(df.values.reshape(480,-1),
                      index=df.index.repeat(480))
    dfc = dfc.replace(727199802211965, np.nan).dropna()
    return dfc.values

In [ ]:
def run_dtw(df1, df2):
    dist, cost, acc, path = dtw(get_dataframe_column_for_comparison(df1), get_dataframe_column_for_comparison(df2), dist=lambda x, y: norm(x - y, ord=1))
    return dist

In [ ]:
X = sad.iloc[:, :-1].values#sample_dataframe.iloc[:, :-1].values
y = sad.iloc[:, -1].values#sample_dataframe.iloc[:, -1].values

In [ ]:
X

In [ ]:
y

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42, stratify=y)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1, metric=run_dtw)

In [ ]:
knn.fit(X_train, y_train)

In [ ]:
# train_accuracy[i] = knn.score(X_train, y_train)

In [ ]:
#Compute accuracy on the test set
# test_accuracy[i] = knn.score(X_test, y_test) 

In [ ]:
y_pred= knn.predict(X_test)

In [ ]:
accuracy_score(y_pred,y_test)

In [ ]:
print("Precision: {}%".format(100*precision_score(y_test, y_pred, average="weighted")))
print("Recall: {}%".format(100*recall_score(y_test, y_pred, average="weighted")))
print("f1_score: {}%".format(100*f1_score(y_test, y_pred, average="weighted")))

In [ ]:
print("")
print("Confusion Matrix:")
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)
normalised_confusion_matrix = np.array(confusion_matrix, dtype=np.float32)/np.sum(confusion_matrix)*100

In [ ]:
# edit this as per the file name 
LABELS = ['LT', 'RT', 'BFS', 'LRS', 'RLS', 'FBS']

In [ ]:
# Plot Results:
width = 12
height = 12
plt.figure(figsize=(width, height))
plt.imshow(
    normalised_confusion_matrix,
    interpolation='nearest',
    cmap=plt.cm.rainbow
)
plt.title("Confusion matrix \n(normalised to % of total test data)")
plt.colorbar()
tick_marks = np.arange(8)
plt.xticks(tick_marks, LABELS, rotation=90)
plt.yticks(tick_marks, LABELS)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.savefig('cf_matrix')
plt.show()

In [ ]:
# Plot Results:
width = 12
height = 12
plt.figure(figsize=(width, height))
plt.imshow(
    confusion_matrix,
    interpolation='nearest',
    cmap=plt.cm.rainbow
)
plt.title("Confusion matrix \n(non-normalised to % of total test data)")
plt.colorbar()
tick_marks = np.arange(8)
plt.xticks(tick_marks, LABELS, rotation=90)
plt.yticks(tick_marks, LABELS)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
# plt.savefig('cf_matrix')
plt.show()

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
# Compute confusion matrix
# cnf_matrix = confusion_matrix(y_test, y_pred)
# np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(confusion_matrix, classes=LABELS,
                      title='Confusion matrix, without normalization')

In [ ]:
# scores = cross_val_score(knn, X, y, cv=54)

In [ ]:
# scores

In [ ]:
# print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))